# 区間推定

**区間推定**とは、データを用いて未知母数Θの値の存在範囲を区間として推定する方法である。  
たとえば、標準正規分布$N(0,1)$の上側2.5%点、および上側97.5%点から以下の確率式が成り立つ
$$
P(-1.96 \leq μ \leq 1.96) = 0.95
$$
この時の確率0.95は**信頼率**あるいは**信頼係数**とよび、標本に依存して得られる区間を**信頼区間**、信頼区間の上限と下限を**信頼限界**とよぶ。 
信頼率を大きくすると、信頼区間がμを含む確率は大きくなるが、区間幅が大きくなりμの値を推測する上での有用性を失う。  
信頼率を一定に保ったまま区間幅を大きくするには、標本サイズnを大きくすればよい。

In [12]:
# 母平均の信頼区間を求めてみる
import math
import pandas as pd
from scipy.stats import norm
from sklearn.datasets import load_iris

iris = load_iris()
iris_df = pd.DataFrame(
        iris.data,
        columns=iris.feature_names,
    )

sl_df_num =100
sl_df = iris_df["sepal length (cm)"]
sl_sample_df = sl_df.sample(n=sl_df_num) # 100このデータをサンプルとして使用

p_var = sl_df.var() # 母分散
s_mean = sl_sample_df.mean() # 標本平均


bottom, up = norm.interval(0.95, loc=s_mean, scale=math.sqrt(p_var/sl_df_num))
print(f"母平均μの95%信頼区間：{bottom:.3f}<=μ<={up:.3f}")

母平均μの95%信頼区間：5.685<=μ<=6.009


# 分散の区間推定

確率変数$X$の母集団分布が正規分布$N(\mu, \sigma^2)$である場合の母分散$\sigma^2$の区間推定について考える。ここで、母平均$\mu$は未知とする。  
母集団からの独立な標本$X_1, X_2,.., X_n$の標本平均$\bar{X}$からの偏差平方和を$T^2=\sum_{i=1}^{n} (X_i - \bar{X})^2$とすると、$\chi^2 = \frac{T^2}{\sigma^2}$は自由度n-1のカイ二乗分布に従う。  
母平均の区間推定と同様に、カイ二乗分布の上側2.5%点$\chi^2_{0.025}(n-1)$および上側97.5%点$\chi^2_{0.975}(n-1)$から、以下の確率式が成り立つ。
$$
P(\chi^2_{0.975}(n-1) \leq \chi^2 \leq \chi^2_{0.025}(n-1)) = 0.95
$$
上式に$\chi^2 = T^2 / \sigma^2$を代入し、$\sigma^2$について整理すると、
$$
P(\frac{T^2}{\chi^2_{0.025}(n-1)} \leq \chi^2 \leq \frac{T^2}{\chi^2_{0.975}(n-1)}) = 0.95
$$
となる。

In [13]:
# 分散の区間推定を実行
import numpy as np
from scipy.stats import chi2

sigma_hat2 = np.var(sl_sample_df, ddof=1)

chi2_lower, chi2_upper = chi2.interval(0.95, df=sl_df_num-1)
print(f"母分散の95%信頼区間：{sl_df_num*sigma_hat2/chi2_lower:.3f}<=σ^2<={sl_df_num*sigma_hat2/chi2_upper:.3f}")

母分散の95%信頼区間：1.155<=σ^2<=0.660


# 分散の比の区間推定

$X_{11},X_{12},...,X_{1n_1}$が、互いに正規分布$N(\mu_1, \sigma_1^2)$に従い、さらにそれとは独立に$X_{21},X_{22},...,X_{2n_2}$が、互いに独立に正規分布$N(\mu_2, \sigma_2^2)$に従うとき、統計量$F = \frac{V_1/\sigma_1^2}{V_2/\sigma_2^2}$の自由度$(n_1-1, n_2-1)$の$F$分布に従う。  
ここで、$V_1 = \frac{1}{n_1-1} \sum_{i=1}^{n_1} (X_{1i}-\bar{X}_1)^2$、$V_2 = \frac{1}{n_2-1} \sum_{i=1}^{n_2} (X_{2i}-\bar{X}_2)^2$、$\bar{X}_1 = \frac{1}{n_1} \sum_{i=1}^{n_1} X_{1i}$、$\bar{X}_2 = \frac{1}{n_2} \sum_{i=1}^{n_2} X_{2i}$である。  
自由度$(\phi_1, \phi_2)$の$F$分布の上側2.5%および上側97.5%点をそれぞれ$F_{0.025}(\phi_1, \phi_2), F_{0.975}(\phi_1, \phi_2)$とすると、$\phi_1 = n_1-1, \phi_2 = n_2-1$として以下の確率式が成り立つ。
$$
P(F_{0.975}(\phi_1, \phi_2) \leq F \leq F_{0.025}(\phi_1, \phi_2)) = 0.95
$$
上式に$F = \frac{V_1/\sigma_1^2}{V_2/\sigma_2^2}$を代入し、$\frac{\sigma_1^2}{\sigma_2^2}$について整理すると、
$$
P(\frac{V_1}{V_2} \cdot \frac{1}{F_{0.025}(\phi_1, \phi_2)} \leq F \leq \frac{V_1}{V_2} \cdot \frac{1}{F_{0.975}(\phi_1, \phi_2)}) = 0.95
$$
となる。

In [14]:
# 分散の比の区間推定を実行
from scipy.stats import f

sw_df = iris_df["sepal width (cm)"]
sw_df_num = 100
sw_sample_df = sw_df.sample(n=sw_df_num) # 100このデータをサンプルとして使用

sl_sigma_hat2 = np.var(sl_sample_df, ddof=1)
sw_sigma_hat2 = np.var(sw_sample_df, ddof=1)

f_lower, f_upper = f.interval(0.95, dfn=sl_df_num-1, dfd=sw_df_num-1)
print(f"母比率の比の95%信頼区間：{sl_sigma_hat2/sw_sigma_hat2*1/f_lower:.3f}<=σ1^2/σ2^2<={sl_sigma_hat2/sw_sigma_hat2*1/f_upper:.3f}")

母比率の比の95%信頼区間：7.718<=σ1^2/σ2^2<=3.494


# 多項分布の信頼区間

確率$p_i$で事象$A_i(i=1,2,...,k)$が起こるような試行を$n$回行ったとき、それぞれの事象が起こる回数$N_i$に関する確率分布を多項分布という。  
ある事象$A_i$が起こるかどうかに着目すると、多項分布は二項分布に帰着するため、平均は$E[N_i]=np_i$、分散は$V[N_i]=np_i(1-p_i)$となる。  
ここで$\hat{p}_i = \frac{N_i}{n}$とし、$u_i = \frac{\hat{p}_i-p_i}{\sqrt{\hat{p}_i(1-\hat{p}_i)/n}}$とすると、$u_i$は漸近的に標準正規分布に従い、
$$
p(-1.96 \leq u_i \leq 1.96) = 0.95
$$
が成り立つ。  
$u_i$の右辺を代入し、$p_i$について整理すると、
$$
P(\hat{p}_i - 1.96 \sqrt{\hat{p}_i(1-\hat{p}_i)/n} \leq p_i \leq \hat{p}_i + 1.96 \sqrt{\hat{p}_i(1-\hat{p}_i)/n}) = 0.95
$$
となり、多項分布の95%信頼区間$P(\hat{p}_i - 1.96 \sqrt{\hat{p}_i(1-\hat{p}_i)/n}, \quad \hat{p}_i + 1.96 \sqrt{\hat{p}_i(1-\hat{p}_i)/n})$が得られる。

In [15]:
# サイコロを例に多項分布の区間推定を実行
import random

dice = list(range(1,7))
dice_selected = 1
n_all = 1000
n1 = random.choices(dice, k=n_all).count(dice_selected)

p1_hat = n1/n_all
ui_denominator = math.sqrt(p1_hat*(1-p1_hat)/n_all)

multi1_lower, multi1_upper = norm.interval(0.95, loc=0, scale=1)
print(f"pi(サイコロ1)の信頼区間：{p1_hat+multi1_lower*ui_denominator:.3f}<=pi<={p1_hat+multi1_upper*ui_denominator:.3f}")

pi(サイコロ1)の信頼区間：0.157<=pi<=0.205


# 多項分布の差の信頼区間

多項分布に従う確率変数$N_i(i=1,2,...,k)$について、$N_1$と$N_2$の共分散は、負の値をとる。つまり事象$A_1$が起こる回数が増えれば、事象$A_2$が起こる回数が減ることを意味している。    
ここで$p_1-p_2$の信頼区間を求めるために、$\hat{p_1}-\hat{p_2} = \frac{N_1}{n}-\frac{N_2}{n}$の期待値と分散を考える。  
$$
E[\hat{p_1}-\hat{p_2}] = p_1 - p_2
$$
$$
V[\hat{p_1}-\hat{p_2}] = \frac{p_1(1-p_1)}{n} - \frac{p_2(1-p_2)}{n} + \frac{2p_1 p_2}{n} 
$$
となる。
$$
\mu = \frac{(\hat{p_1}-\hat{p_2})-(p_1 - p_2)}{\sqrt{\frac{\hat{p}_1(1-\hat{p}_1)}{n} - \frac{\hat{p}_2(1-\hat{p}_2)}{n} + \frac{2\hat{p}_1 \hat{p}_2}{n}}}
$$
とすると、$\mu$は漸近的に標準正規分布に従うため、95%信頼区間は
$$
((\hat{p_1}-\hat{p_2})-1.96\sqrt{\frac{\hat{p}_1(1-\hat{p}_1)}{n} - \frac{\hat{p}_2(1-\hat{p}_2)}{n} + \frac{2\hat{p}_1 \hat{p}_2}{n}}, \quad \\  \quad 
(\hat{p_1}-\hat{p_2})+1.96\sqrt{\frac{\hat{p}_1(1-\hat{p}_1)}{n} - \frac{\hat{p}_2(1-\hat{p}_2)}{n} + \frac{2\hat{p}_1 \hat{p}_2}{n}})
$$
となる

In [16]:
# 多項分布の差の区間推定を実行
dice_selected = 2
n2 = random.choices(dice, k=n_all).count(dice_selected)
p2_hat = n2/n_all

u_denominator = math.sqrt(
    p1_hat*(1-p1_hat)/n_all+\
    p2_hat*(1-p2_hat)/n_all+\
    2*p1_hat*p2_hat/n_all
)

print(f"p1-p2の95%信頼区間：{(p1_hat-p2_hat)+multi1_lower*u_denominator:.3f}<=p1-p2<={(p1_hat-p2_hat)+multi1_upper*u_denominator:.3f}")


p1-p2の95%信頼区間：0.001<=p1-p2<=0.071
